In [1]:
from dfttopif import directory_to_pif
pif = directory_to_pif("/home/maxhutch/science/alloy.pbe/B/B.hR12")

In [2]:
from pypif.util.read_view import ReadView
rv = ReadView(pif)
print(rv.keys())
print(rv.chemical_formula)

dict_keys(['Converged', 'Density Functional Theory', 'XC Functional', 'Pressure', 'VASP', 'Cutoff Energy', 'Relaxed', 'k-Points per Reciprocal Atom', 'Total Energy', 'Psuedopotentials'])
B12


# Working with PIFs

Writing analysis and post-processing on top of PIFs lets us:
 1. Pull in data from collaborators and published sources
 1. Share our methodology with other researchers with data in PIFs

## `pypif` package

The `pypif` package provides two APIs:
 1. Low level read/write API
 1. High[er] leve read-only API

### Low level API

The low level API closely mirrors the PIF schema definition

In [3]:
print("The total energy is {}".format(
        next(x for x in pif.properties if x.name == "Total Energy")
        .scalars[0].value
     ))

The total energy is -80.346476


Note that the `name` is a field in a properties object, so we are searching for the first property with the `name` "Total Energy".

### High[er] level API: ReadView

It might be more natural to index the property by its name.  We provide a `ReadView` that wraps the PIF and provides such an index:

In [4]:
from pypif.util.read_view import ReadView
rv = ReadView(pif)
print("The total energy is {}".format(
        rv["Total Energy"].scalars[0].value
    ))

The total energy is -80.346476


The `scalars` member of the property contains the value of the property.  We also have access to metadata associated with that value, e.g. units:

In [5]:
print("The total energy is {} {}".format(
        rv["Total Energy"].scalars[0].value,
        rv["Total Energy"].units)
     )

The total energy is -80.346476 eV
